In [ ]:
!pip install flair


     |████████████████████████████████| 450kB 12.7MB/s 
     |████████████████████████████████| 798kB 44.3MB/s 
     |████████████████████████████████| 19.7MB 82.8MB/s 
     |████████████████████████████████| 71kB 12.6MB/s 
     |████████████████████████████████| 983kB 49.5MB/s 
     |████████████████████████████████| 1.1MB 49.5MB/s 
     |████████████████████████████████| 1.3MB 45.1MB/s 
     |████████████████████████████████| 890kB 46.9MB/s 
     |████████████████████████████████| 2.9MB 44.7MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=7eb7b417eba4ee4b7223fa970623b0c01d4a400f9a4faa47318209fa8a25e593
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45613 sha256=9c6c823ba5275c4719a6b5643fb63f1bca3af939ffb0dc59641176e54be08394
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57feff

In [ ]:
from flair.data import Sentence

# Make a sentence object by passing an untokenized string and the 'use_tokenizer' flag
sentence = Sentence('The grass is green.', use_tokenizer=True)

# Print the object to see what's in there
print(sentence)

Sentence: "The grass is green ."   [− Tokens: 5]


In [ ]:
for token in sentence:
    print(token)
    

Token: 1 The
Token: 2 grass
Token: 3 is
Token: 4 green
Token: 5 .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from flair.datasets import ColumnCorpus
from flair.data import Corpus
from flair.embeddings import WordEmbeddings, BytePairEmbeddings, StackedEmbeddings, CharacterEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence
from flair.embeddings import StackedEmbeddings

In [ ]:
def corpus_loader(data_path):
  columns = {0: 'text', 1: 'ner'}
  corpus: Corpus = ColumnCorpus(data_path, columns, train_file='outBIOtrain-conll.txt', test_file='outBIOtest-conll.txt')
  
  return corpus

In [ ]:
def get_embedding(param):
  embedding_types = [ WordEmbeddings('/content/drive/MyDrive/wvec models/wvec_model.wv'),
  CharacterEmbeddings(),
  WordEmbeddings('tr'),
  TransformerWordEmbeddings('dbmdz/bert-base-turkish-cased', layers='-1')]
  if (param=='w2vec'):    
    return embedding_types[0]
  elif (param=='char'):    
    return embedding_types[1]
  elif (param=='fsttx'):   
    return embedding_types[2]
  elif (param=='transformer'):    
    return embedding_types[3]
  elif (param=='w2vec+char'):
    return StackedEmbeddings([embedding_types[0],embedding_types[1]])
  elif (param=='fsttx+char'):
    return StackedEmbeddings([embedding_types[2],embedding_types[1]])
  elif (param=='w2vec+transformer'):
    return StackedEmbeddings([embedding_types[0],embedding_types[3]])
  elif (param=='fsttx+transformer'):
    return StackedEmbeddings([embedding_types[2],embedding_types[3]])
  elif (param=='w2vec+char+transformer'):
    return StackedEmbeddings([embedding_types[0],embedding_types[1],embedding_types[3]])
  elif (param=='fsttx+char+transformer'):
    return StackedEmbeddings([embedding_types[2],embedding_types[1],embedding_types[3]])        


In [ ]:
def main():
  corpus = corpus_loader("/content/drive/MyDrive/ner_data")
  tag_dictionary = corpus.make_tag_dictionary(tag_type='ner')
  embedding = get_embedding('fsttx+char+transformer')
  tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=embedding,
                                       tag_dictionary=tag_dictionary,
                                       tag_type='ner',
                                       use_crf=True)
  trainer : ModelTrainer = ModelTrainer(tagger, corpus)
  trainer.train('resources/taggers/example-ner',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=50)

In [ ]:
if __name__ == '__main__':
    main()

2020-12-06 19:25:32,896 Reading data from /content/drive/MyDrive/ner_data
2020-12-06 19:25:32,898 Train: /content/drive/MyDrive/ner_data/outBIOtrain-conll.txt
2020-12-06 19:25:32,899 Dev: None
2020-12-06 19:25:32,900 Test: /content/drive/MyDrive/ner_data/outBIOtest-conll.txt
2020-12-06 19:26:32,827 ----------------------------------------------------------------------------------------------------
2020-12-06 19:26:32,830 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('tr')
    (list_embedding_1): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_2): TransformerWordEmbeddings(
      (model): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(32000, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,)